In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 32.0)
cnt = resample_cnt(cnt, 64.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)

In [ ]:
x_right = train_set.X[train_set.y == 0]

x_rest = train_set.X[train_set.y == 2]

inputs_a = np_to_var(x_right[:160,0:1,:,None], dtype=np.float32)

inputs_b = np_to_var(x_rest[:160,0:1,:,None], dtype=np.float32)
inputs = [inputs_a, inputs_b]

In [ ]:
cnt_bandpassed =  mne_apply(
    lambda a: bandpass_cnt(a, 8, 13, cnt.info['sfreq'],
                           filt_order=10,
                           axis=1), cnt)

In [ ]:
alpha_set = create_signal_target_from_raw_mne(cnt_bandpassed, marker_def, ival)
x_alpha_right = alpha_set.X[alpha_set.y == 0]

x_alpha_rest = alpha_set.X[alpha_set.y == 2]

alpha_a = np_to_var(x_alpha_right[:160,0:1,:,None], dtype=np.float32)

alpha_b = np_to_var(x_alpha_rest[:160,0:1,:,None], dtype=np.float32)
inputs_alpha = [alpha_a, alpha_b]

In [ ]:
fig, axes = plt.subplots(80,4, figsize=(14,80), sharex=True, sharey=True)
for i_class in range(len(inputs)):
    for i_example in range(len(inputs_alpha[i_class])):
        i_row = i_example // 2
        i_col = i_example % 2
        i_col += i_class * 2
        axes[i_row][i_col].plot(var_to_np(inputs_alpha[i_class][i_example]).squeeze(),
                               color=seaborn.color_palette()[i_class])
fig.suptitle('Input signals', y=0.9)

from matplotlib.lines import Line2D
lines = [Line2D([0], [0], color=seaborn.color_palette()[i_class],) for i_class in range(2)]
labels = ['Right', 'Rest',]
axes[0][-1].legend(lines, labels, bbox_to_anchor=(1,1,0,0))

In [ ]:
bps = np.abs(np.fft.rfft(cnt_bandpassed.get_data()[0,8000:]))

In [ ]:
freqs = np.fft.rfftfreq(n=cnt_bandpassed.get_data()[0,8000:].shape[0], d=1/64.0)
plt.plot(freqs, bps)
print(freqs[np.argsort(bps)[-1]])

In [ ]:
plt.plot(np.mean(var_to_np(inputs_alpha[0]), axis=0).squeeze())
plt.plot(np.mean(var_to_np(inputs_alpha[1]), axis=0).squeeze())
plt.plot(np.sin(np.linspace(0,11*2*np.pi,64)) * 0.1)

In [ ]:
freq = th.ones(1, requires_grad=True)
freq.data[:] = 11

#th.sin(th.linspace(0,2*np.pi*freq))

In [ ]:
phase = th.rand(1) * 2*np.pi

step = 64 / (2*np.pi * freq)

template = th.sin(th.linspace(0,63,64) * step + phase)

timecourse = nn.functional.avg_pool1d(th.rand(64).unsqueeze(0).unsqueeze(0) * 2,9,stride=1,padding=4
                             ).squeeze()

plt.plot(var_to_np(timecourse))
plt.plot(var_to_np(timecourse * template))

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)),
         
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True))
    )
    

In [ ]:
from reversible.util import set_random_seeds
set_random_seeds(2019011641, True)
feature_model = nn.Sequential(
    rev_block(8,32),
    rev_block(8,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 16 x 4
    rev_block(16,32),
    rev_block(16,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 32 x 2
    rev_block(32,32),
    rev_block(32,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 64 x 1
    rev_block(64,64),
    rev_block(64,64),
    ViewAs((-1,64,1,1), (-1,64)),
)

n_clusters = len(inputs)
n_dims = inputs[0].shape[2]
means_per_cluster = [th.autograd.Variable(th.ones(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]

amp_mean = th.autograd.Variable(th.ones(1), requires_grad=True)
log_amp_std = th.autograd.Variable(th.zeros(1), requires_grad=True)

# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]
freq = th.ones(1, requires_grad=True)
freq.data[:] = 11



from copy import deepcopy
optimizer = th.optim.Adam(
                          [
    {'params': list(feature_model.parameters()),
    'lr': 1e-3,
    'weight_decay': 0},], betas=(0,0.9))

optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster + [freq, amp_mean, log_amp_std],
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))


def w_init(m):
    if hasattr(m, 'weight'):
        nn.init.zeros_(m.weight)

feature_model.apply(w_init);

In [ ]:
def create_in_samples(n_samples):
    samples = get_gauss_samples(n_samples, mean, std, truncate_to=3)

    phase = th.rand(n_samples) * 2 * np.pi

    step = 64 / (2*np.pi * freq)

    amps = get_gauss_samples(n_samples, amp_mean, th.exp(log_amp_std))
    
    inverted = invert(feature_model, samples)
    mean_inverted = th.mean(inverted, dim=1, keepdim=True)
    interpolated = nn.functional.interpolate(mean_inverted, size=(64,1), mode='bilinear', align_corners=False)
    timecourses = interpolated.squeeze()
    timecourses = th.sigmoid(timecourses)
    timecourses = timecourses / th.mean(timecourses, dim=1, keepdim=True)
    template = th.sin((th.linspace(0,63,64) * step).unsqueeze(0) + phase.unsqueeze(1))
    examples = amps * template  * timecourses 
    return examples

In [ ]:
from reversible.gaussian import get_gauss_samples

n_epochs = 2500
for i_epoch in range(n_epochs):
    mean = means_per_cluster[0]
    std = th.exp(stds_per_cluster[0])
    n_samples = 640
    examples = create_in_samples(n_samples)
    sliced_loss = sliced_from_samples(inputs_alpha[0].squeeze(), examples, 20, None)

    optimizer.zero_grad()
    optim_dist.zero_grad()

    sliced_loss.backward()

    optimizer.step()
    optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Sliced: {:.2f}".format(sliced_loss.item()))
        print("Freq {:.2f}".format(freq.item()))
        print("Mean", means_per_cluster[0])
        print("Std", std)
        print("AmpMean", amp_mean)
        print("AmpStd", th.exp(log_amp_std))
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(std).squeeze())
        plt.title("Standard deviations latent space")
        display(fig)
        plt.close(fig)
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(examples[:5]).T)
        display(fig)
        plt.close(fig)
        
        n_samples = 3200
        in_samples = var_to_np(create_in_samples(n_samples)).squeeze()

        this_ins = var_to_np(inputs_alpha[0].squeeze())

        diffs = this_ins[:,None] - in_samples[None]

        diffs = np.sqrt(np.sum(np.square(diffs), axis=-1))

        import ot
        coupling = ot.emd([],[], diffs)
        mask = coupling > (1/(2*len(samples)))
        assert np.sum(mask)  == len(samples)
        argmaxes = np.argmax(mask, axis=0)


        fig, axes = plt.subplots(10,4, figsize=(16,20), sharex=True, sharey=True)
        for i_in, ax in enumerate(axes.flatten()):
            i_samples = np.nonzero(argmaxes == i_in)
            ax.plot(in_samples[i_samples].T, color=seaborn.color_palette()[0], lw=0.5)
            ax.plot(this_ins[i_in], color='black')
        display(fig)
        plt.close(fig)

In [ ]:
# multi scale
# as latent distribution: mean over all
# mean over split // 2 
# mean over split // 4
# mean over split // 8
# ...
# for phase and amplitude


In [ ]:
mean = means_per_cluster[0]
std = th.exp(stds_per_cluster[0])
i_dims = np.argsort(var_to_np(std))[::-1][:2]

In [ ]:
def invert_samples(samples):
    phase = th.zeros(1)
    step = 64 / (2*np.pi * freq)
    amps = amp_mean#get_gauss_samples(n_samples, amp_mean, th.exp(log_amp_std))
    timecourses = invert_to_timecourse(samples)
    template = th.sin((th.linspace(0,63,64) * step).unsqueeze(0) + phase.unsqueeze(1))
    examples = amps * template  * timecourses 
    return examples

def invert_to_timecourse(samples):
    inverted = invert(feature_model, samples)
    mean_inverted = th.mean(inverted, dim=1, keepdim=True)
    interpolated = nn.functional.interpolate(mean_inverted, size=(64,1), mode='bilinear', align_corners=False)
    timecourses = interpolated.squeeze(-1).squeeze(1)
    timecourses = th.sigmoid(timecourses)
    timecourses = timecourses / th.mean(timecourses, dim=1, keepdim=True)
    return timecourses

In [ ]:
n_f_vals = 15
f_vals_0 = th.linspace((mean[i_dims[0]] - 1 * std[i_dims[0]]).item(),
                       (mean[i_dims[0]] + 1 * std[i_dims[0]]).item(),
                       n_f_vals)

f_vals_1 = th.linspace((mean[i_dims[1]] - 1 * std[i_dims[1]]).item(),
                       (mean[i_dims[1]] + 1 * std[i_dims[1]]).item(),
                       n_f_vals)
curves = np.zeros((n_f_vals, n_f_vals, 64))
with th.no_grad():
    for i_f_val_0, f_val_0 in enumerate(f_vals_0):
        for i_f_val_1, f_val_1 in enumerate(f_vals_1):
            this = mean.clone().detach()
            this.data[i_dims[0]] = f_val_0
            this.data[i_dims[1]] = f_val_1
            inverted = invert_to_timecourse(this.unsqueeze(0))
            curves[i_f_val_0, i_f_val_1] = var_to_np(inverted).squeeze()
fig, axes = plt.subplots(n_f_vals, n_f_vals,figsize=(18,18), sharex=True, sharey=True)
for i_row, i_col in itertools.product(range(n_f_vals), range(n_f_vals)):
    axes[i_row, i_col].plot(curves[i_row, i_col])
display(fig)
plt.close(fig)

In [ ]:
curves = np.zeros((n_f_vals, n_f_vals, 64))
with th.no_grad():
    for i_f_val_0, f_val_0 in enumerate(f_vals_0):
        for i_f_val_1, f_val_1 in enumerate(f_vals_1):
            this = mean.clone().detach()
            this.data[i_dims[0]] = f_val_0
            this.data[i_dims[1]] = f_val_1
            inverted = invert_samples(this.unsqueeze(0))
            curves[i_f_val_0, i_f_val_1] = var_to_np(inverted).squeeze()
fig, axes = plt.subplots(n_f_vals, n_f_vals,figsize=(14,14), sharex=True, sharey=True)
for i_row, i_col in itertools.product(range(n_f_vals), range(n_f_vals)):
    axes[i_row, i_col].plot(curves[i_row, i_col])
display(fig)
plt.close(fig)

In [ ]:
import itertools
list(itertools.product(range(3), range(3)))

In [ ]:
f_vals_

In [ ]:
f_vals_0

In [ ]:
in_samples.shape

In [ ]:
plt.plot(np.mean(np.abs(np.fft.rfft(var_to_np(inputs_alpha[1]).squeeze())), axis=0))